# Извлечение ключевых слов

## Часть первая. Датасет

Нам нужны тексты с ключевыми словами. Мы можем взять датасет со статьями с хабра. Статьи длмнные, но обычно посвящены одной широкой теме или нескольким узким. Возможно, эти данные будет проще анализировать, так как ресурс знакомый и читаемый. Ключевые слова есть, они по большей части описывают использованные технологии или темы
<br>
Датасет взят вот отсюда: https://github.com/mannefedov/ru_kw_eval_datasets

In [1]:
#!pip install jsonlines

In [2]:
import re
from tqdm import tqdm
import jsonlines

linelist = []
with jsonlines.open('habr_texts.jsonlines') as f:
    for obj in f:
        linelist.append(obj)

In [3]:
#len(linelist)

In [4]:
#print(linelist[0])

## Часть вторая. Ручная разметка и эталон

In [5]:
tokens = 0
texts = []
keywords_origin = []
for i in range(5):
    texts.append(linelist[i]['content'])
    keywords_origin.append(linelist[i]['keywords'])
    tokens += len(linelist[i]['content'].split())
print(tokens)

13085


Токенов хватает

In [6]:
keywords_of_markup = []
keywords_of_markup.append(['MassTransit', 'open source', '.NET', 'команды', 'события'])
keywords_of_markup.append(['XenForo', 'плагины', 'форумные движки', 'геймификация', 'форум'])
keywords_of_markup.append(['Postgresql', 'failover', 'standby', 'master', 'repmgr', 'кластер'])
keywords_of_markup.append(['SQL', 'синтаксический анализатор', 'транслятор', '1C', 'Irony'])
keywords_of_markup.append(['аудиотехнологии', 'амплитудно-частотные характеристики', 'музыка', 'аудиосистемы', 'наушники'])

In [7]:
import pandas as pd
dataset_of_texts = pd.DataFrame()
dataset_of_texts['text'] = texts
dataset_of_texts['original_keywords'] = keywords_origin
dataset_of_texts['markup_keywords'] = keywords_of_markup
dataset_of_texts

,text,original_keywords,markup_keywords
0,"MassTransit это open source библиотека, разра...","[.net, rabbitmq, masstransit]","[MassTransit, open source, .NET, команды, собы..."
1,Введение и выбор решения \r\nРано или поздно ...,"[геймификация, xenforo, форумные движки, форум...","[XenForo, плагины, форумные движки, геймификац..."
2,\r\nНа сегодняшний день процедура реализации ...,"[postgresq, haproxy, pgbouncer, keepalived, re...","[Postgresql, failover, standby, master, repmgr..."
3,"Как часто, программируя очередную бизнес-фичу,...","[irony, .net, c#, грамматический разбор, синта...","[SQL, синтаксический анализатор, транслятор, 1..."
4,"Индустрия звука, о которая была у всех на слух...","[аудиомания, мифы и реальность, акустика, ауди...","[аудиотехнологии, амплитудно-частотные характе..."


С одной стороны, размеченные и изначальные ключевые слова похожи и местами совпадают, но не везде. Например, одно и то же слово может быть написано капсом или нет, или это могут быть одинаковые термины из русского и английского. Размеченные слова следуют в основном за технической стороной статьи, тогда как оригинальные могут быть довольно широкими: например, "мифы и реальность" или "бухгалтеры и программисты". Кроме того, в исходной разметке больше слов, даже если они очень похожи и на ту же тему. Кажется, что наилучшим исходом будет объединение ключевых слов. Итак, объединим.

In [8]:
keywords_standard = []
for i in range(5):
    keywords_standard.append(set(keywords_origin[i]) | set(keywords_of_markup[i]))
dataset_of_texts['standard_keywords'] = keywords_standard

In [9]:
dataset_of_texts

,text,original_keywords,markup_keywords,standard_keywords
0,"MassTransit это open source библиотека, разра...","[.net, rabbitmq, masstransit]","[MassTransit, open source, .NET, команды, собы...","{события, MassTransit, masstransit, open sourc..."
1,Введение и выбор решения \r\nРано или поздно ...,"[геймификация, xenforo, форумные движки, форум...","[XenForo, плагины, форумные движки, геймификац...","{геймификация, форум, плагины, форумные движки..."
2,\r\nНа сегодняшний день процедура реализации ...,"[postgresq, haproxy, pgbouncer, keepalived, re...","[Postgresql, failover, standby, master, repmgr...","{postgresq, replication, keepalived, pgbouncer..."
3,"Как часто, программируя очередную бизнес-фичу,...","[irony, .net, c#, грамматический разбор, синта...","[SQL, синтаксический анализатор, транслятор, 1...","{грамматический парсер, синтаксический анализа..."
4,"Индустрия звука, о которая была у всех на слух...","[аудиомания, мифы и реальность, акустика, ауди...","[аудиотехнологии, амплитудно-частотные характе...","{аудиотехника, музыка, аудиомания, аудиотехнол..."


Чудесно, всё объединилось.  А теперь можно попробовать методы автоматического ввыделения ключевых слов

## Часть третья. Автоматическое выделение ключевых слов

Для каждого метода поставлено ограничение по весу слова, чтобы в ключевые слов не попадало слишком много. Выставлено приблизительным подбором, чтобы слов в каждый текст попадало не слишком много. Можно было бы сказать "обязательно по 10 слов", например, но кажется, что так лучше -- на хабре у разных статей разное количество ключевых слов

### Препроцессинг

И здесь-то нас и подстерегают все сложности, которые есть у этого датасета. Он одновременно на русском, с использованием некоторых английских терминов, которые при этом являются основополагающими, да ещё и с добавлением кода, который, конечно, вряд ли должен быть в ключевых словах -- но вот незадача, он местами очень похож на английский! Так что ждут нас проблемы и наши тексты весьма сложно формализуемы.

In [10]:
import stanza
#stanza.download('ru')
#stanza.download('en')

nlp_ru = stanza.Pipeline(lang='ru', processors='tokenize,lemma,pos')
nlp_en = stanza.Pipeline(lang='en', processors='tokenize,lemma,pos')

2021-11-07 21:38:43 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |

2021-11-07 21:38:43 INFO: Use device: cpu
2021-11-07 21:38:43 INFO: Loading: tokenize
2021-11-07 21:38:43 INFO: Loading: pos
2021-11-07 21:38:46 INFO: Loading: lemma
2021-11-07 21:38:47 INFO: Done loading processors!
2021-11-07 21:38:47 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |

2021-11-07 21:38:47 INFO: Use device: cpu
2021-11-07 21:38:47 INFO: Loading: tokenize
2021-11-07 21:38:47 INFO: Loading: pos
2021-11-07 21:38:50 INFO: Loading: lemma
2021-11-07 21:38:50 INFO: Done loading processors!


In [11]:
def prepocess(text, nlp):
    doc = nlp(text)
    stops = ['ADP', 'AUX', 'CCONJ', 'DET', 'INTJ', 'PART', 'PUNCT', 'SCONJ']
    final_text = []
    for sent in doc.sentences:
        sent_list = []
        count_foreing = 0
        for word in sent.words:
            #print(word)
            if word.upos not in stops:
                if not re.search('[1234567890\\\/&{}_]+?', word.text):
                    sent_list.append(word.lemma.lower())
                if word.feats:
                    if 'Foreign=Yes' in word.feats:
                        count_foreing += 1
                    
        if count_foreing < 10:
            final_text.extend(sent_list)
    return ' '.join(final_text)

Что тут, собственно, происходит.  Мы берём текст, и для каждого предложения смотрим, а не код ли оно. Если в предложении меньше десяти английских слов, то ладно, не код.
Мы можем так задеть какие-то части предложений или, наоборот, оставить маленькие кусочки кода, но в целом ситуация должна стать лучше

In [12]:
#будет небыстро, это stanza
clean_texts = []
for text in tqdm(dataset_of_texts['text']):
    clean_texts.append(prepocess(text, nlp_ru))
clean_keywords = []
for stand_keywords in dataset_of_texts['standard_keywords']:
    words = []
    for word in stand_keywords:
        preprocessed_word = prepocess(word, nlp_ru)
        if preprocessed_word != '':
            words.append(preprocessed_word)
    clean_keywords.append(words)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [04:41<00:00, 56.23s/it]


In [13]:
clean_texts_keywords = pd.DataFrame()
clean_texts_keywords['texts'] = clean_texts
clean_texts_keywords['standard_keywords'] = clean_keywords

### RAKE

In [14]:
import RAKE
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords

In [15]:
stop = stopwords.words('russian')

In [16]:
rake = RAKE.Rake(stop)

In [17]:
rake_res = []
for text in clean_texts_keywords['texts']:
    text_res = []
    kw_list = rake.run(text, maxWords=2, minFrequency=1)
    for el in kw_list:
        if el[1] > 0:
            text_res.append(el)
    rake_res.append(text_res)
clean_texts_keywords['rake'] = rake_res
rake_res

[[('получить сообщение', 4.0),
  ('выбор консьюмер', 4.0),
  ('статья рассказать', 4.0),
  ('newline console', 4.0),
  ('находиться имплементация', 4.0),
  ('схожий имплементация', 4.0),
  ('write', 1.0),
  ('environment', 1.0),
  ('start', 1.0)],
 [('бывать коммерческий', 4.0),
  ('поле низко', 4.0),
  ('интерес представлять', 4.0),
  ('создавать включая', 4.0),
  ('важный первый', 4.0),
  ('определить трофей', 4.0),
  ('пробовать оценить', 4.0),
  ('далее', 1.0),
  ('таба', 1.0),
  ('комбинировать', 1.0),
  ('видный', 1.0),
  ('детально', 1.0)],
 [('мастер-нод восстановление', 9.0),
  ('интернет вроде', 4.0),
  ('это необходимый', 4.0),
  ('мочь оказаться', 4.0),
  ('хост выдавать', 4.0),
  ('conf postgresql', 4.0),
  ('grep repmgrd', 4.0),
  ('sh telegram', 4.0),
  ('sh сохраниться', 4.0),
  ('установить заранее', 4.0),
  ('ip addr', 4.0),
  ('конфига видно', 4.0),
  ('перейти подключение', 4.0),
  ('настраивать ip', 4.0),
  ('крона например', 4.0),
  ('нода например', 3.5),
  ('вид

### TextRank

In [18]:
from summa import keywords

In [19]:
textrank_res = []
for text in clean_texts_keywords['texts']:
    text_res = []
    kw_list = keywords.keywords(text, language='russian', additional_stopwords=stop, scores=True)
    for el in kw_list:
        if el[1] > 0.08:
            text_res.append(el)
    textrank_res.append(text_res)
clean_texts_keywords['textrank'] = textrank_res
textrank_res

[[('сообщение', 0.37372780733073396),
  ('консьюмер', 0.21694473153211707),
  ('тип', 0.19349401048401066),
  ('команда событие', 0.19002909437854149),
  ('exchange', 0.178654008862924),
  ('использовать', 0.1508095810045571),
  ('наименование', 0.13086391570503952),
  ('процесс который', 0.1261816464383145),
  ('код', 0.11518649109884217),
  ('случай', 0.10800879901922156),
  ('конфигурация контейнер', 0.1033046983403988),
  ('masstransit', 0.10244018287209909),
  ('следующий', 0.09473694013866121),
  ('документация', 0.09048382505464365),
  ('обработка', 0.09001512272063948),
  ('метод publish интерфейс', 0.08980450439736458),
  ('данные', 0.08885155106174701),
  ('данный', 0.08885155106174701),
  ('bus', 0.08142726608297576),
  ('await', 0.08131099763786405),
  ('имя', 0.08053156440669765),
  ('качество message', 0.08002436234844718)],
 [('система трофей', 0.3204399314731482),
  ('пользователь', 0.20697203966901426),
  ('badge', 0.16267262333176122),
  ('плагина', 0.1548200247713522

### Tf-Idf

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [49]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True, norm='l2')
X_tfidf = tfidf_vectorizer.fit_transform(clean_texts_keywords['texts']).toarray()
vocabulary = tfidf_vectorizer.get_feature_names()

tfidf_result = []
for words_of_text in X_tfidf:
    text_words = []
    for i, word_w in enumerate(words_of_text):
        if word_w > 0.1:
            text_words.append((vocabulary[i], word_w))
    tfidf_result.append(text_words)
clean_texts_keywords['tfidf'] = tfidf_result

tfidf_result

[[('endpoint', 0.10748105307330351),
  ('exchange', 0.21496210614660702),
  ('masstransit', 0.1343513163416294),
  ('publish', 0.10748105307330351),
  ('send', 0.10748105307330351),
  ('интерфейс', 0.10748105307330351),
  ('команда', 0.2339392701319992),
  ('консьюмер', 0.2687026326832588),
  ('контейнер', 0.10748105307330351),
  ('конфигурация', 0.1083937960797493),
  ('наименование', 0.1343513163416294),
  ('они', 0.10596766334253696),
  ('отправка', 0.1083937960797493),
  ('смс', 0.10748105307330351),
  ('событие', 0.23846635137544847),
  ('сообщение', 0.4986114619668468),
  ('тип', 0.15138237620362424)],
 [('badge', 0.24377269691433537),
  ('level', 0.12188634845716768),
  ('step', 0.1044740129632866),
  ('xenforo', 0.1044740129632866),
  ('геймификация', 0.1044740129632866),
  ('достижение', 0.1044740129632866),
  ('количество', 0.1044740129632866),
  ('критерий', 0.2089480259265732),
  ('пользователь', 0.16857795167380396),
  ('система', 0.15695689742947014),
  ('сообщество', 0.1

## Часть четвёртая. Морфологические и синтаксические шаблоны

Давайте посмотрим на наши эталонные списки

In [22]:
for i in range(5):
    print(clean_texts_keywords['standard_keywords'][i])

['событие', 'masstransit', 'masstransit', 'open source', '.net', 'rabbitms', 'net', 'команда']
['геймификация', 'форум', 'плагин', 'форумный движок', 'xenforo', 'gamification', 'xenforo', 'привлечение пользователь']
['postgresq', 'replication', 'keepalived', 'pgbouncer', 'cluster', 'ha', 'standby', 'haproxy', 'master', 'postgresql', 'кластер', 'failover', 'repmgr']
['грамматический парсер', 'синтаксический анализатор', 'синтаксический анализ', 'irony', '.net', 'бухгалтерия программист', 'sql', 'анализатор код', 'бухгалтерия', 'грамматический разбор', 'регулярный выражение', 'грамматика', 'кт', 'regexp', 'sql', 'транслятор', 'irony']
['аудиотехника', 'музыка', 'аудиомание', 'аудиотехнология', 'акустика', 'аудиосистема', 'амплитудный частотный характеристика', 'наушник', 'миф реальность']


Вроде видно паттерны. Основные -- Noun, Adj + Noun, PNoun, Noun + Noun. Однако это информация на глаз, а хорошо бы проверить с помощью такого же парсера, как и тот, которым будут размечаться тексты. Есть проблема -- русский и английский вместе. Было бы здорово, конечно, использовать какую-нибудь мультиязычную модель, но сначала давайте попробуем что-нибудь попроще. Если бы перед нами стоялазадача обработать текст, было бы, конечно, сложнее. Здесь можно попробовать всё, что содержит кириллицу, отдать парсеру для русского языка, а всё, что содержит латиницу -- парсеру для английского

In [28]:
def is_pattern(text, nlp, patterns):
    doc = nlp(text)
    stops = ['ADP', 'AUX', 'CCONJ', 'DET', 'INTJ', 'PART', 'PUNCT', 'SCONJ']
    pattern = []
    for sent in doc.sentences:
        for word in sent.words:
            if re.search('[\\\/&{}]+?', word.text):
                break
            else:
                if  word.upos not in stops:
                    pattern.append(word.upos)
    #print(pattern)          
    if pattern in patterns:
        return True
    else:
        return False

In [24]:
pattern_res = {}
patterns_en = [['PROPN'], ['PROPN', 'PROPN'], ['NOUN']]
patterns_ru = [['ADJ', 'NOUN'], ['NOUN', 'NOUN'], ['ADJ', 'ADJ', 'NOUN'], ['NOUN']]

In [25]:
def pattern_keywords(keywords, nlp_en=nlp_en, patterns_en=patterns_en, nlp_ru=nlp_ru, patterns_ru=patterns_ru):
    all_text_filter = []
    for text_keywords in keywords:
        text_keywords_filter = []
        for word in text_keywords:
            flag = False
            if re.search('[A-Za-z]+?', word[0]):
                flag = is_pattern(word[0], nlp_en, patterns_en)
            if re.search('[А-Яа-яЁё]+?', word[0]):
                flag = is_pattern(word[0], nlp_ru, patterns_ru)
            if flag is True:
                #print(word)
                text_keywords_filter.append(word)
        all_text_filter.append(text_keywords_filter)
    return all_text_filter

In [29]:
clean_texts_keywords['rake_patterns'] = pattern_keywords(clean_texts_keywords['rake'])
clean_texts_keywords['rake_patterns']

0    [(выбор консьюмер, 4.0), (схожий имплементация...
1                                        [(таба, 1.0)]
2    [(conf postgresql, 4.0), (grep repmgrd, 4.0), ...
3    [(таблица бд, 4.0), (бессонный цейтнот, 4.0), ...
4    [(сайт аудиомания, 4.0), (полный катушка, 4.0)...
Name: rake_patterns, dtype: object

In [30]:
clean_texts_keywords['textrank_patterns'] = pattern_keywords(clean_texts_keywords['textrank'])
clean_texts_keywords['textrank_patterns']

0    [(сообщение, 0.37372780733073396), (консьюмер,...
1    [(система трофей, 0.3204399314731482), (пользо...
2    [(нода, 0.30096812238258275), (ноды, 0.3009681...
3    [(узел, 0.1687500170866756), (кода, 0.12131693...
4    [(система, 0.19009798864268804), (мочь, 0.1723...
Name: textrank_patterns, dtype: object

In [31]:
clean_texts_keywords['tfidf_patterns'] = pattern_keywords(clean_texts_keywords['tfidf'])
clean_texts_keywords['tfidf_patterns']

0    [(endpoint, 0.10748105307330351), (exchange, 0...
1    [(badge, 0.24377269691433537), (level, 0.12188...
2    [(pghost, 0.16333476828697313), (postgres, 0.2...
3    [(ast, 0.2959134022754096), (irony, 0.10292640...
4    [(аудиомания, 0.10532637462606463), (аудиосист...
Name: tfidf_patterns, dtype: object

## Часть пятая. Точность, полнота, F-мера

In [32]:
def precision(tp, fp):
    return tp / (tp + fp)    


def recall(tp, fn):
    return tp / (tp + fn)


def f_score(tp, fp, fn):
    p = precision(tp, fp)
    r = recall(tp, fn)
    return 2*(p*r)/(p+r)

In [33]:
def count_results(standard_keywords, auto_keywords):
    true_positive = 0
    false_positive = 0
    false_negative = 0
    for i, text in enumerate(auto_keywords):
        for word in text:
            if word[0] in standard_keywords[i]:
                true_positive += 1
            else:
                false_positive += 1
    for i, text in enumerate(standard_keywords):
        for word in text:
            if word not in [lemm_tuple[0] for lemm_tuple in auto_keywords[i]]:
                false_negative += 1
            
    #print(true_positive, false_positive, false_negative)
    if true_positive != 0:
        print('precision:', precision(true_positive, false_positive))
        print('recall:', recall(true_positive, false_negative))
        print('f_score:', f_score(true_positive, false_positive, false_negative))
    else:
        print('Нет правильных ответов')    
    
    return true_positive, false_positive, false_negative

### Снова эталон

Отфильтруем из него только те слова и сочетания, которые есть в текстах -- потому что иначе откуда их нашим алгоритмам брать

In [34]:
keys_from_text = []
for i in range(5):
    print(i)
    one_text_keys = []
    for word in clean_texts_keywords['standard_keywords'][i]:
        if word in clean_texts_keywords['texts'][i]:
            one_text_keys.append(word)
        else:
            print(word)
    keys_from_text.append(set(one_text_keys))

0
open source
.net
rabbitms
1
форумный движок
gamification
привлечение пользователь
2
replication
3
грамматический парсер
.net
бухгалтерия программист
анализатор код
грамматический разбор
4
аудиомание
миф реальность


И здесь мы видим, что хоть stanza и хороша, некоторые слова она клоняет неправильно. Посмотрите на "аудиомание" -- это название сайта, "аудиомания", и оно будет в ключевых словах

In [35]:
clean_texts_keywords['keywords_from_text'] = keys_from_text
keys_from_text

[{'masstransit', 'net', 'команда', 'событие'},
 {'xenforo', 'геймификация', 'плагин', 'форум'},
 {'cluster',
  'failover',
  'ha',
  'haproxy',
  'keepalived',
  'master',
  'pgbouncer',
  'postgresq',
  'postgresql',
  'repmgr',
  'standby',
  'кластер'},
 {'irony',
  'regexp',
  'sql',
  'бухгалтерия',
  'грамматика',
  'кт',
  'регулярный выражение',
  'синтаксический анализ',
  'синтаксический анализатор',
  'транслятор'},
 {'акустика',
  'амплитудный частотный характеристика',
  'аудиосистема',
  'аудиотехника',
  'аудиотехнология',
  'музыка',
  'наушник'}]

### Метрики без грамматических фильтров

In [36]:
clean_texts_keywords

,texts,standard_keywords,rake,textrank,tfidf,rake_patterns,textrank_patterns,tfidf_patterns,keywords_from_text
0,качество message broker мочь выступать rabbitm...,"[событие, masstransit, masstransit, open sourc...","[(получить сообщение, 4.0), (выбор консьюмер, ...","[(сообщение, 0.37372780733073396), (консьюмер,...","[(endpoint, 0.10748105307330351), (exchange, 0...","[(выбор консьюмер, 4.0), (схожий имплементация...","[(сообщение, 0.37372780733073396), (консьюмер,...","[(endpoint, 0.10748105307330351), (exchange, 0...","{masstransit, событие, net, команда}"
1,введение выбор решение рано поздно наступать м...,"[геймификация, форум, плагин, форумный движок,...","[(бывать коммерческий, 4.0), (поле низко, 4.0)...","[(система трофей, 0.3204399314731482), (пользо...","[(badge, 0.24377269691433537), (level, 0.12188...","[(таба, 1.0)]","[(система трофей, 0.3204399314731482), (пользо...","[(badge, 0.24377269691433537), (level, 0.12188...","{xenforo, форум, плагин, геймификация}"
2,сегодняшний день процедура реализация «failove...,"[postgresq, replication, keepalived, pgbouncer...","[(мастер-нод восстановление, 9.0), (интернет в...","[(нода, 0.30096812238258275), (ноды, 0.3009681...","[(pghost, 0.16333476828697313), (postgres, 0.2...","[(conf postgresql, 4.0), (grep repmgrd, 4.0), ...","[(нода, 0.30096812238258275), (ноды, 0.3009681...","[(pghost, 0.16333476828697313), (postgres, 0.2...","{postgresq, keepalived, pgbouncer, cluster, st..."
3,часто программировать очередной бизнес фича вы...,"[грамматический парсер, синтаксический анализа...","[(сводиться перекладывание, 4.0), (таблица бд,...","[(узел, 0.1687500170866756), (язык запрос тран...","[(ast, 0.2959134022754096), (irony, 0.10292640...","[(таблица бд, 4.0), (бессонный цейтнот, 4.0), ...","[(узел, 0.1687500170866756), (кода, 0.12131693...","[(ast, 0.2959134022754096), (irony, 0.10292640...","{синтаксический анализатор, транслятор, синтак..."
4,индустрия звук который все слух практически по...,"[аудиотехника, музыка, аудиомание, аудиотехнол...","[(сайт аудиомания, 4.0), (усилить особенно, 4....","[(звук который, 0.2044702034863903), (система,...","[(акустический, 0.10532637462606463), (аудиома...","[(сайт аудиомания, 4.0), (полный катушка, 4.0)...","[(система, 0.19009798864268804), (мочь, 0.1723...","[(аудиомания, 0.10532637462606463), (аудиосист...","{аудиотехника, музыка, амплитудный частотный х..."


In [37]:
count_results(clean_texts_keywords['keywords_from_text'], clean_texts_keywords['rake'])

Нет правильных ответов


(0, 103, 37)

In [38]:
count_results(clean_texts_keywords['keywords_from_text'], clean_texts_keywords['textrank'])

precision: 0.05172413793103448
recall: 0.16216216216216217
f_score: 0.0784313725490196


(6, 110, 31)

In [51]:
count_results(clean_texts_keywords['keywords_from_text'], clean_texts_keywords['tfidf'])

precision: 0.19767441860465115
recall: 0.4594594594594595
f_score: 0.2764227642276423


(17, 69, 20)

### Метрики с грамматическими фильтрами

In [40]:
count_results(clean_texts_keywords['keywords_from_text'], clean_texts_keywords['rake_patterns'])

Нет правильных ответов


(0, 32, 37)

In [41]:
count_results(clean_texts_keywords['keywords_from_text'], clean_texts_keywords['textrank_patterns'])

precision: 0.09090909090909091
recall: 0.16216216216216217
f_score: 0.11650485436893203


(6, 60, 31)

In [42]:
count_results(clean_texts_keywords['keywords_from_text'], clean_texts_keywords['tfidf_patterns'])

precision: 0.25757575757575757
recall: 0.4594594594594595
f_score: 0.3300970873786408


(17, 49, 20)

Улучшились!

In [43]:
for i in clean_texts_keywords['keywords_from_text']:
    print(i)

{'masstransit', 'событие', 'net', 'команда'}
{'xenforo', 'форум', 'плагин', 'геймификация'}
{'postgresq', 'keepalived', 'pgbouncer', 'cluster', 'standby', 'master', 'haproxy', 'postgresql', 'ha', 'кластер', 'failover', 'repmgr'}
{'синтаксический анализатор', 'транслятор', 'синтаксический анализ', 'кт', 'regexp', 'sql', 'регулярный выражение', 'бухгалтерия', 'грамматика', 'irony'}
{'аудиотехника', 'музыка', 'амплитудный частотный характеристика', 'акустика', 'аудиосистема', 'аудиотехнология', 'наушник'}


In [52]:
for i in clean_texts_keywords['tfidf_patterns']:
    print('************')
    for el in i:
        print(el[0])

************
endpoint
exchange
masstransit
интерфейс
команда
консьюмер
контейнер
конфигурация
наименование
отправка
смс
событие
сообщение
тип
************
badge
level
step
xenforo
геймификация
достижение
количество
критерий
пользователь
система
сообщество
таб
трофей
трофея
уровень
форум
************
pghost
postgres
postgresql
repmgr
standby
su
кластер
мастер
нода
сервер
скрипт
текст
файл
************
ast
irony
regexp
sql
база
грамматика
запрос
метод
правило
свойство
узел
язык
************
аудиомания
аудиосистема
аудиотехника
звук
звучание
мочь
наушник
система
усилитель
устройство
цап


## Часть шестая. Ошибки и их анализ

Итак. Где и кто ошибается. RAKE -- к сожалению, везде. Ни одного верного ключевого слова. Вероятнее всего, он просто мало подходит к нашим текстам, так как они довольно специфические. TextRank -- меньше, чем tf-idf, но фильтры ему помогли. 
Tf-Idf из них трёх показал лучший скор по всем параметрам, особенно с фильтрами.
<br>
Почему?
<br>
У нас очень специфический датасет. Это не новости, не повествовательные последовательные тексты, не описания событий. Это тьюториалы и статьи с хабра. Во-первых, другой жанр -- многио тьюториалов и пояснений к коду. Во-вторых, куски кода, которые плохо отделяются от основного текста. В-третьих, тьма терминов на английском прямо посреди русского текста. Даже мультиязычная модель нам здесь, возможно, не помощник -- так как английские слова являются или названиями, или терминами. Вероятно, нужно уулучшить очистку текста от кода -- например, с помощью регулярок (хотя это сложно, на все-то языки программирования) или применения дополнительных парсеров/знаний о синтаксисе, что тоже не очень просто.  Почему мы выбрали такую очистку? У нас, конечно, 5 статей, можно было бы вручную всё удалить, но если применять на большем -- не пойдёт. Самым разумным методом избавления от кода было бы собирание либо отдельно, либо с разделителями, но это надо данные ещё раз собирать. Ещё можно было бы просто выкидывать куски латиницы, которые перемежаются любым синтаксисом, длиной более 50, скажем, символов. Тоже могло сработать, тоже не везде.
<br>
Если мы посмотрим на выделенные, например, тф-идфом слова, то увидим, что в них много синонимов/слов из близких областей. Например, в пятом тексте это звук и звучание. А в пользовательских тегах вместо этого -- музыка, немного другое, более общее слово не из технических характеристик. В целом, ключевые слова и термины выделяются, но естественно, алгоритмам не очень ясно, у кого из них более широкая семантика, а у кого более узкая, или, например, что вот эта библиотека -- для вот такого языка, или мы рассматриваем один из группы плагинов. В этом случае в пользовательских словах скорее будут более обширные темы, чем более частные, а в авторматичеом выделении и те, и другие. Или, например, мы используем какую-то библиотеку в помощь другой, как пандас при выделении ключевых слов. Тогда в ключевых словах пользователя ожидается упоминание основной библиотеки, но вряд ли всех, используемых "в помощь". А вот при одинаковом количестве употреблений алгоритм разницу может и не понять.
<br>
Также существует проблема модели для обработки текста. Stanza в принципе неплоха, но где-то ломается и портит результаты и она. Решение -- попробовать разные модели и, возможно, выбрать другую